In [15]:
using Test

In [16]:
f = readlines("day14.input");

In [17]:
function parse_int64(num)
    parse(Int64, num)
end

parse_int64 (generic function with 1 method)

In [18]:
function parse_input(input)
    min_x, max_x = 10000, 0
    min_y, max_y = 10000, 0

    rocks = Vector{Vector{Tuple{Int64, Int64}}}()
    for line in input
        rock = Vector{Tuple{Int64, Int64}}()
        for coord in split(line, "->")
            x, y = map(parse_int64, split(coord, ","))
            push!(rock, (x, y))
            max_x = max(x, max_x)
            max_y = max(y, max_y)
            min_y = min(y, min_y)
            min_x = min(x, min_x)

        end
        push!(rocks, rock)
    end

    rocks, (min_x, max_x), (min_y, max_y)
end

@test parse_input(String.(split("498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9", "\n"))) == ([[(498, 4), (498, 6), (496, 6)], [(503, 4), (502, 4), (502, 9), (494, 9)]], (494, 503), (4, 9))


Test Passed

In [19]:
function travel(p₀, p₁)
    diff = p₁ .- p₀
    step = sum(diff) > 0 ? 1 : -1
    if diff[1] != 0
        return [(x, p₀[2]) for x=p₀[1]:step:p₁[1]]
    end
    return [(p₀[1], y) for y=p₀[2]:step:p₁[2]]
end

travel (generic function with 1 method)

In [20]:
function build_world(rocks)
    world = Dict{Tuple{Int64, Int64}, Int64}()

    for rock in rocks
        for i=1:length(rock)-1
            from, to = rock[i], rock[i+1]
            for p in travel(from, to)
                world[p] = 1
            end
        end
    end

    world
end

build_world (generic function with 1 method)

In [21]:
world = build_world(parse_input(String.(split("498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9", "\n")))[1])

Dict{Tuple{Int64, Int64}, Int64} with 20 entries:
  (496, 9) => 1
  (498, 9) => 1
  (502, 7) => 1
  (499, 9) => 1
  (498, 5) => 1
  (502, 8) => 1
  (496, 6) => 1
  (501, 9) => 1
  (498, 6) => 1
  (497, 9) => 1
  (503, 4) => 1
  (502, 4) => 1
  (497, 6) => 1
  (494, 9) => 1
  (502, 9) => 1
  (500, 9) => 1
  (495, 9) => 1
  (502, 5) => 1
  (498, 4) => 1
  (502, 6) => 1

In [22]:
function drop_sand(world, sand, floor)
    x, y = sand
    while y <= floor
        if !((x, y+1) in keys(world))
            y += 1
            continue
        else
            if !((x-1, y+1) in keys(world))
                y += 1
                x -= 1
                continue
            end
            if !((x+1, y+1) in keys(world))
                y += 1
                x += 1
                continue
            end
        end

        world[(x, y)] = 2
        return true
    end

    return false
end

@test drop_sand(Dict(), (500, 0), 10) == false

Test Passed

In [29]:
function drop_sand_part_2(world, sand, floor)
    x, y = sand
    while !(sand in keys(world))
        if !((x, y+1) in keys(world) && y+1 < floor)
            y += 1
            continue
        else
            if !((x-1, y+1) in keys(world))
                y += 1
                x -= 1
                continue
            end
            if !((x+1, y+1) in keys(world))
                y += 1
                x += 1
                continue
            end
        end

        world[(x, y)] = 2
        return true
    end

    return false
end

drop_sand_part_2 (generic function with 1 method)

In [24]:
function solve_part_1(input)
    rocks, x_range, y_range = parse_input(input)
    floor = y_range[2]
    world = build_world(rocks)
    dropped = 0
    while true
        if !drop_sand(world, (500, 0), floor)
            return dropped
        end
        dropped += 1
    end
end

solve_part_1 (generic function with 1 method)

In [25]:
function solve_part_2(input)
    rocks, x_range, y_range = parse_input(input)
    floor = y_range[2]+2
    world = build_world(rocks)
    for i=500-floor:500+floor
        world[i, floor] = 1
    end
    dropped = 0
    while true
        if !drop_sand(world, (500, 0), floor+1)
            return dropped
        end
        dropped += 1
    end
end

solve_part_2 (generic function with 1 method)

In [26]:
@test solve_part_1(String.(split("498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9", "\n"))) == 24


Test Passed

In [27]:
(solve_part_1(f), solve_part_2(f))

MethodError: MethodError: no method matching !(::Tuple{Int64, Int64})
Closest candidates are:
  !(!Matched::Function) at operators.jl:1077
  !(!Matched::Bool) at bool.jl:35
  !(!Matched::Missing) at missing.jl:101

In [30]:
@test solve_part_2(String.(split("498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9", "\n"))) == 93